In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
from models.ddpm_unet_cattn import SPVUnet
import torch
import lightning as L
from models.g_spvd import GSPVD

In [3]:
from torch.utils.data import DataLoader
from dataloaders.shapenet.shapenet_loader import ShapeNet

categories = ['chair']
path = "../data/ShapeNet"

test_dataset = ShapeNet(path, "test", 2048, categories, load_renders=True)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=4)

Loading (test) renders for chair (03001627):   0%|          | 0/1317 [00:00<?, ?it/s]

In [4]:
from utils.hyperparams import load_hyperparams

hparams_path = f'../checkpoints/distillation/GSPVD/{"-".join(categories)}/hparams.yaml'

hparams = load_hyperparams(hparams_path)

In [5]:
model_args = {
    'voxel_size' : hparams['voxel_size'],
    'nfs' : hparams['nfs'], 
    'attn_chans' : hparams['attn_chans'], 
    'attn_start' : hparams['attn_start'], 
    'cross_attn_chans' : hparams['cross_attn_chans'], 
    'cross_attn_start' : hparams['cross_attn_start'], 
    'cross_attn_cond_dim' : hparams['cross_attn_cond_dim'],
}

model = SPVUnet(**model_args)
model = GSPVD(model=model)

In [6]:
steps = 500
on_all = True
scheduler = 'ddpm'
ckpt_path = f'../checkpoints/distillation/GSPVD/{"-".join(categories)}/{steps}-steps.ckpt'
ckpt = torch.load(ckpt_path, weights_only=False)
model.load_state_dict(ckpt)

<All keys matched successfully>

In [7]:
model = model.cuda().eval()

In [8]:
from my_schedulers.ddpm_scheduler import DDPMSparseScheduler
from my_schedulers.ddim_scheduler import DDIMSparseScheduler
from utils.visualization import display_pointclouds_grid

if scheduler == 'ddim':
    sched = DDIMSparseScheduler(
        beta_min=hparams['beta_min'], 
        beta_max=hparams['beta_max'], 
        steps=steps, 
        init_steps=hparams['n_steps'],
        mode=hparams['mode'],
    )
else:
    sched = DDPMSparseScheduler(
        beta_min=hparams['beta_min'], 
        beta_max=hparams['beta_max'], 
        steps=steps, 
        init_steps=hparams['n_steps'],
        mode=hparams['mode'],
    )

In [9]:
# from schedulers import create_sparse_scheduler
# sched = create_sparse_scheduler()

# gen_pc = sched.sample(model, B, cond_emb=features, mode="conditional").cuda()

In [10]:
from tqdm.auto import tqdm

all_ref_pc = []
all_gen_pc = []

mean = torch.tensor(test_loader.dataset.mean).cuda()
std = torch.tensor(test_loader.dataset.std).cuda()

i = 0
for datapoint in tqdm(test_loader):
    i += 1
    if i > 5 and not on_all:
        continue

    ref_pc = datapoint['pc'].cuda()
    features = datapoint['render-features'].cuda()

    B, N, C = ref_pc.shape
    gen_pc = sched.sample(model, B, N, reference=features)

    all_ref_pc.append(ref_pc)
    all_gen_pc.append(gen_pc)

all_ref_pc = torch.cat(all_ref_pc).cuda()
all_gen_pc = torch.cat(all_gen_pc).cuda()

  0%|          | 0/42 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

...

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

In [11]:
from metrics.evaluation_metrics import compute_all_metrics, jsd_between_point_cloud_sets as JSD
from pprint import pprint

In [12]:
results = compute_all_metrics(all_ref_pc, all_gen_pc, batch_size=32)
results = {k: (v.cpu().detach().item()
               if not isinstance(v, float) else v) for k, v in results.items()}

pprint(results)

  0%|          | 0/1317 [00:00<?, ?it/s]

  0%|          | 0/1317 [00:00<?, ?it/s]

  0%|          | 0/1317 [00:00<?, ?it/s]

{'1-NN-CD-acc': 0.7627183198928833,
 '1-NN-CD-acc_f': 0.6309794783592224,
 '1-NN-CD-acc_t': 0.8944571018218994,
 '1-NN-EMD-acc': 0.8595292568206787,
 '1-NN-EMD-acc_f': 0.8033409118652344,
 '1-NN-EMD-acc_t': 0.9157175421714783,
 'lgan_cov-CD': 0.4517843723297119,
 'lgan_cov-EMD': 0.2999240756034851,
 'lgan_mmd-CD': 0.061506591737270355,
 'lgan_mmd-EMD': 0.44174984097480774,
 'lgan_mmd_smp-CD': 0.10328605026006699,
 'lgan_mmd_smp-EMD': 1.0958306789398193}


In [13]:
jsd = JSD(all_gen_pc.cpu().numpy(), all_ref_pc.cpu().numpy())
results['JSD'] = jsd
pprint('JSD: {}'.format(jsd))

'JSD: 0.015253316957151597'


In [14]:
import json
import os

folder = f'../metrics/{"-".join(categories)}/{scheduler}/no-norm'
os.makedirs(folder, exist_ok=True)

file = os.path.join(folder, f'{steps}-steps.json')
with open(file, 'w') as f:
    json.dump(results, f, indent=4)

In [15]:
def normalize_to_unit_sphere(batched_points: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    """
    Normalize a batched tensor of 3D points to the unit sphere.
    
    Args:
        batched_points: (B, N, 3) tensor, where B = batch size, N = num points.
        eps: Small value to avoid division by zero.
    
    Returns:
        (B, N, 3) tensor, where all points lie within or on the unit sphere.
    """
    # Center points by subtracting their mean (centroid)
    centroid = torch.mean(batched_points, dim=1, keepdim=True)  # (B, 1, 3)
    centered = batched_points - centroid  # (B, N, 3)

    # Find the maximum distance from the origin for each batch
    max_dist = torch.max(
        torch.sqrt(torch.sum(centered ** 2, dim=-1, keepdim=True)),  # (B, N, 1)
        dim=1, keepdim=True
    ).values  # (B, 1, 1)

    # Normalize by dividing by the maximum distance (+ eps for stability)
    normalized = centered / (max_dist + eps)  # (B, N, 3)

    return normalized

In [16]:
all_gen_pc_norm = normalize_to_unit_sphere(all_gen_pc)
all_ref_pc_norm = normalize_to_unit_sphere(all_ref_pc)

In [17]:
results = compute_all_metrics(all_ref_pc_norm, all_gen_pc_norm, batch_size=32)
results = {k: (v.cpu().detach().item()
               if not isinstance(v, float) else v) for k, v in results.items()}

pprint(results)

  0%|          | 0/1317 [00:00<?, ?it/s]

  0%|          | 0/1317 [00:00<?, ?it/s]

  0%|          | 0/1317 [00:00<?, ?it/s]

{'1-NN-CD-acc': 0.632877767086029,
 '1-NN-CD-acc_f': 0.5049354434013367,
 '1-NN-CD-acc_t': 0.7608200311660767,
 '1-NN-EMD-acc': 0.6127562522888184,
 '1-NN-EMD-acc_f': 0.5558086633682251,
 '1-NN-EMD-acc_t': 0.6697039008140564,
 'lgan_cov-CD': 0.5261958837509155,
 'lgan_cov-EMD': 0.5246772766113281,
 'lgan_mmd-CD': 0.005770683288574219,
 'lgan_mmd-EMD': 0.03282294049859047,
 'lgan_mmd_smp-CD': 0.006671624723821878,
 'lgan_mmd_smp-EMD': 0.035347890108823776}


In [18]:
jsd = JSD(all_gen_pc.cpu().numpy(), all_ref_pc.cpu().numpy())
results['JSD'] = jsd
pprint('JSD: {}'.format(jsd))

'JSD: 0.015253316957151597'


In [19]:
import json
import os

folder = f'../metrics/{"-".join(categories)}/{scheduler}/norm'
os.makedirs(folder, exist_ok=True)

file = os.path.join(folder, f'{steps}-steps.json')
with open(file, 'w') as f:
    json.dump(results, f, indent=4)